In [1]:
import requests, json, time

# files
ChampPickedCounterFile = open("ChampPickedCounter.txt", "w")
champWinRateFile = open("champWinRate.txt", "w")
ChampWinListFile = open("ChampWinList.txt", "w")
ChampLoseListCounterFile = open("ChampLoseListCounter.txt", "w")
GamesInfoFile = open("GamesInfo.txt", "w")

# variables 
Patch = "11.24.1"
MinimumGamesPlayed = 100
Champion = "Evelynn"

In [2]:
# Recupere la list des chall
api_key = "RGAPI-034a4ddc-4862-4229-80cf-face74a491d9"
GetChallPlayers = "https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"+ "?api_key=" + api_key
response1 = requests.get(GetChallPlayers).json()

In [ ]:
# recupere le puuid des chall
print (time.strftime("%H : %M : %S"))
info = (response1['entries'])
playersPuuid = []
playersName = []
for i in range(299):
        GetPlayersPuuid = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/"+info[i]['summonerId']+ "?api_key=" + api_key
        response2 = requests.get(GetPlayersPuuid).json()
        playersPuuid.append(response2["puuid"])
        time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))
# recupere les 20 dernieres games de chaque joueur depuis une certaine date
print (time.strftime("%H : %M : %S"))
PlayersGames = []
date = "1639029600"
for i in range(299):
        GetPlayersGames = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" + playersPuuid[i] + "/ids?startTime=" + date +"&queue=420&type=ranked&start=0&count=20&api_key=" + api_key
        response2 = requests.get(GetPlayersGames).json()
        PlayersGames = PlayersGames + response2
        time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))
PlayersGames.sort()
# verifie que la liste des games a ete trie
print(len(PlayersGames))
PlayersGames = list(set(PlayersGames))
seen = set()
result = []
for item in PlayersGames:
    if item not in seen:
        seen.add(item)
        result.append(item)
print(len(result))
# recupere les infos de chaque game
print (time.strftime("%H : %M : %S"))
GamesInfo = []
PlayersGamesRange = len(PlayersGames) - 1
for i in range(PlayersGamesRange):
    GetGamesInfo = "https://europe.api.riotgames.com/lol/match/v5/matches/" + PlayersGames[i] + "?api_key=" + api_key
    response3 = requests.get(GetGamesInfo).json()
    GamesInfo.append(response3)
    time.sleep(2)
print("end")
print (time.strftime("%H : %M : %S"))


11 : 46 : 02
end
11 : 56 : 40
11 : 56 : 40
end
12 : 07 : 16
2443
1638
12 : 07 : 16


In [8]:
#nb de games par champions
# recupere la liste des champions joue sur toutes les games dispo, et on tri par ordre alphabet
from operator import itemgetter
champsList = []
count = len(GamesInfo) - 1
for j in range (count):
    for i in range (10):
        champsList.append(GamesInfo[j]["info"]["participants"][i]["championName"])
# compte le nombre d apparition d un champion
ChampWinList = [[x,champsList.count(x)] for x in set(champsList)]
ChampPickedCounter = [[x,champsList.count(x)] for x in set(champsList)]
# pour trier par nb de games : , key=itemgetter(1), reverse=True
ChampPickedCounter = sorted(ChampPickedCounter)
ChampWinList = sorted(ChampWinList)
#nb de loose par champions
# recupere la liste des champions joue sur toutes les games dispo, et on tri par ordre alphabet
from operator import itemgetter
ChampLoseList = []
count = len(GamesInfo) - 1
for j in range (count):
    for i in range (10):
        if GamesInfo[j]["info"]["participants"][i]["win"] == False:
            ChampLoseList.append(GamesInfo[j]["info"]["participants"][i]["championName"])
# compte le nombre d apparition d un champion
ChampLoseListCounter = [[x,ChampLoseList.count(x)] for x in set(ChampLoseList)]
# pour trier par nb de games : , key=itemgetter(1), reverse=True
ChampLoseListCounter = sorted(ChampLoseListCounter)
#nb de games loose par champions
import copy
champWinRate = []
countLose =  len(ChampLoseListCounter) - 1
countPick = len(ChampPickedCounter) - 1
# calcul du winrate d un champion 
for i in range (countPick):
    for j in range (countLose):
        if ChampWinList[i][0] == ChampLoseListCounter[j][0]:
            ChampWinList[i][1] = ChampWinList[i][1] - ChampLoseListCounter[j][1]
            winrate = round(ChampWinList[i][1] * 100 / ChampPickedCounter[i][1], 2)
            if ChampPickedCounter[i][1] > MinimumGamesPlayed:
                champWinRate.append([ChampWinList[i][0], winrate])
champWinRate = sorted(champWinRate, key=lambda x: x[1], reverse=True)

In [9]:
# on affiche toutes nos donnees

print("------------Picks-----------------")
print(sorted(ChampPickedCounter, key=lambda x: x[1], reverse=True))
print("-------------Win rate----------------")
print(champWinRate)
print("------------win nb-----------------")
print(ChampWinList)
print("-------------lose nb----------------")
print(ChampLoseListCounter)


# on ecrit les infos dans des fichiers pour pouvoir les stocker
AllGameFile = open("AllGame.json", "w")
AllGameFile.write(json.dumps(GamesInfo))
AllGameFile.close()

ChampPickedCounterFile = open("ChampPickedCounter.json", "w")
ChampPickedCounterFile.write(json.dumps(ChampPickedCounter))
ChampPickedCounterFile.close()

ChampPickedCounterFile = open("ChampPickedCounter.json", "w")
ChampPickedCounterFile.write(json.dumps(ChampPickedCounter))
ChampPickedCounterFile.close()

champWinRateFile = open("champWinRate.json", "w")
champWinRateFile.write(json.dumps(champWinRate))
champWinRateFile.close()

ChampWinListFile = open("ChampWinList.json", "w")
ChampWinListFile.write(json.dumps(ChampWinList))
ChampWinListFile.close()

ChampLoseListCounterFile = open("ChampLoseListCounter.json", "w")
ChampLoseListCounterFile.write(json.dumps(ChampLoseListCounter))
ChampLoseListCounterFile.close()

GamesInfoFile = open("GamesInfo.json", "w")
GamesInfoFile.write(json.dumps(GamesInfo))
GamesInfoFile.close()

------------Picks-----------------
[['LeeSin', 523], ['Viego', 445], ['Jhin', 395], ['Jinx', 387], ['Kaisa', 366], ['Lux', 348], ['Viktor', 330], ['Qiyana', 329], ['Yone', 324], ['Jayce', 311], ['Lulu', 310], ['Ezreal', 288], ['Akshan', 287], ['Caitlyn', 280], ['Zed', 257], ['Karma', 252], ['Akali', 247], ['Talon', 236], ['Vex', 234], ['Graves', 231], ['Rakan', 230], ['Irelia', 221], ['Lucian', 214], ['Thresh', 212], ['Corki', 209], ['Sylas', 206], ['Fiora', 205], ['Nami', 205], ['Janna', 202], ['Syndra', 202], ['Nidalee', 191], ['Yasuo', 185], ['Leona', 181], ['Gragas', 170], ['Leblanc', 165], ['Pyke', 163], ['Ekko', 162], ['XinZhao', 160], ['Draven', 150], ['Nautilus', 150], ['TwistedFate', 147], ['Vayne', 142], ['Kayn', 136], ['Aphelios', 135], ['Camille', 119], ['Twitch', 112], ['Bard', 110], ['Zoe', 108], ['Jax', 107], ['Singed', 107], ['Ashe', 105], ['Kassadin', 104], ['Sona', 103], ['Xerath', 99], ['Riven', 98], ['Orianna', 90], ['Blitzcrank', 89], ['Tryndamere', 86], ['Gwen', 8

In [23]:
# Calcul du nombre d'occurrences des items pour un champion donne
ItemList = []
WinOrNot = True
for e in range(count):
    for i in range(10):
        for j in range(6):
            if GamesInfo[e]["info"]["participants"][i]["championName"] == Champion:
#                 est ce que c est une game win ou non (peut etre mis en commit)
#                 if GamesInfo[e]["info"]["participants"][i]["win"] == TrueOrFalse:
                    ItemList.append(GamesInfo[e]["info"]["participants"][i]["item" + str(j)])
print(ItemList)
ItemListcounter = [[x,ItemList.count(x)] for x in set(ItemList)]
ItemListcounter = sorted(ItemListcounter, key=lambda x: x[1], reverse=True)

# on change l'alias des items pour leur nom
GetItemInfo = requests.get('https://ddragon.leagueoflegends.com/cdn/'+ Patch +'/data/en_US/item.json').json()
for i in range(len(ItemListcounter)):
    if ItemListcounter[i][0] != 0:
        number = str(ItemListcounter[i][0])
        ItemListcounter[i][0] = GetItemInfo["data"][str(number)]["name"]
    if ItemListcounter[i][0] == 0:
        ItemListcounter[i][0] = "null"
        
print(ItemListcounter)

[3135, 4632, 3152, 1082, 3020, 3108, 3089, 3041, 3152, 3020, 3100, 1026, 3152, 4645, 3089, 3135, 1052, 3020, 3041, 3135, 3152, 3020, 3089, 0, 3152, 4645, 4630, 3020, 3089, 0, 3152, 3041, 0, 0, 0, 3020, 4645, 0, 3152, 3135, 3089, 3020, 3152, 3089, 3135, 3020, 3102, 0, 3020, 3152, 4645, 1082, 0, 3135, 1058, 3041, 3152, 3020, 3135, 0, 3152, 3108, 3041, 3020, 3089, 1052, 3152, 1026, 3020, 1082, 3102, 4630, 3152, 3020, 3089, 3145, 1082, 0, 3152, 3041, 3102, 3020, 4630, 3089, 3102, 3100, 3152, 3089, 3020, 3135, 3152, 3102, 3041, 3020, 0, 0, 3020, 3100, 3152, 3089, 1026, 3041, 3020, 2031, 3041, 3089, 0, 3152, 1058, 2031, 3152, 3041, 3020, 4645, 3152, 3100, 4629, 3102, 3089, 3041, 2055, 2031, 3152, 3041, 3089, 3020, 4645, 3135, 3152, 3089, 4629, 3020, 3152, 3020, 3041, 3089, 3135, 1058, 3152, 3100, 3089, 0, 3102, 3020, 3152, 3089, 3041, 3135, 3020, 3108, 3020, 3135, 3152, 3041, 0, 1028, 3152, 1026, 3089, 0, 3041, 3020, 2422, 2031, 3152, 3041, 1052, 0, 3152, 3102, 4630, 3041, 3020, 0, 3152, 203

In [108]:
# calcul du winrate par rapport au premier objectif
GamesFirstObjWin = 0
Obj = "inhibitor"
GameWin = True
FirstObj = True
for i in range(len(GamesInfo)):
    for e in range(2):
        if GamesInfo[i]["info"]["teams"][e]["win"] == GameWin and GamesInfo[i]["info"]["teams"][e]["objectives"][Obj]["first"] == FirstObj:
            GamesFirstObjWin = GamesFirstObjWin + 1
GamesFirstObjWinRate = GamesFirstObjWin * 100  / len(GamesInfo)
GamesFirstObjWinRate = round(GamesFirstObjWinRate, 2)
print("Winrate with",Obj,"killed first",GamesFirstObjWinRate)

Winrate with inhibitor killed first 73.97


In [107]:
# creation d un dictionaire avec la key et le nom du champion
bannedchamp = []
GetChampName = requests.get('https://ddragon.leagueoflegends.com/cdn/'+ Patch +'/data/en_US/champion.json').json()
ChampNameList = []
for k in GetChampName["data"]:
    ChampNameList.append(k)
ChampListKey = []
for i in range(len(ChampNameList)):
    ChampListKey.append([ChampNameList[i], GetChampName["data"][ChampNameList[i]]["key"]])

# calcul du nombre de ban d un champion
for f in range(len(GamesInfo)):
    for e in range(2):
        for i in range(5):
            if GamesInfo[f]["info"]["teams"][e]["bans"][i]["championId"] != -1:
                bannedchamp.append(GamesInfo[f]["info"]["teams"][e]["bans"][i]["championId"])
bannedchamp = [[x,bannedchamp.count(x)] for x in set(bannedchamp)]
bannedchamp = sorted(bannedchamp, key=lambda x: x[1], reverse=True)

# on change la key par le nom du champion
for i in range(len(bannedchamp)):
    for e in range(len(ChampListKey)):
        if str(bannedchamp[i][0]) == ChampListKey[e][1]:
            bannedchamp[i][0] = ChampListKey[e][0]
print("Most banned champ :", bannedchamp)

Most banned champ : [['Talon', 1227], ['Akshan', 946], ['Yuumi', 881], ['Kassadin', 773], ['Viktor', 650], ['Qiyana', 629], ['Irelia', 590], ['Jax', 472], ['Draven', 470], ['Yone', 444], ['Viego', 443], ['Fiora', 437], ['Lulu', 388], ['LeeSin', 386], ['Leblanc', 369], ['Zed', 337], ['Vex', 291], ['Evelynn', 284], ['Tryndamere', 265], ['Katarina', 250], ['Jinx', 222], ['Rengar', 169], ['Camille', 164], ['Thresh', 159], ['Jayce', 148], ['Lucian', 137], ['Karthus', 131], ['Akali', 127], ['Bard', 114], ['TwistedFate', 109], ['Yasuo', 109], ['Kayn', 91], ['Jhin', 89], ['Lux', 87], ['Nami', 85], ['Nidalee', 79], ['Hecarim', 77], ['Singed', 76], ['Vayne', 76], ['Nunu', 75], ['Corki', 71], ['Darius', 70], ['Kaisa', 70], ['Karma', 68], ['Malphite', 68], ['Khazix', 68], ['Leona', 65], ['Sona', 63], ['TahmKench', 58], ['Riven', 54], ['Caitlyn', 53], ['Xerath', 53], ['Zilean', 52], ['XinZhao', 48], ['Pyke', 45], ['Blitzcrank', 45], ['Xayah', 45], ['Nautilus', 44], ['Anivia', 42], ['Sett', 40], ['G

In [105]:
WinBlueCount = 0

LoseBlueCount = 0

for i in range(len(GamesInfo)):
    if GamesInfo[i]["info"]["teams"][0]["win"] == True:
        WinBlueCount = WinBlueCount + 1
    elif GamesInfo[i]["info"]["teams"][0]["win"] == False:
        LoseBlueCount = LoseBlueCount + 1

print("Blue side win : ",WinBlueCount)

print("Blue side lose : ",LoseBlueCount)


Blue side win :  815
Blue side lose :  791


In [127]:
# calcul du gain d exp et de gold des supports
SupportGoldEarned = []
SupportExp = []
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "SUPPORT":
            SupportGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])


for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "SUPPORT":
            SupportExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])

# calcul du gain d exp et de gold des jungler
JunglerGoldEarned = []
JunglerExp = []
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "JUNGLE":
            JunglerGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])


for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["individualPosition"] == "JUNGLE":
            JunglerExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])
            
# calcul du gain d exp et de gold des jungler
CarryGoldEarned = []
CarryExp = []
for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "CARRY":
            CarryGoldEarned.append(GamesInfo[i]["info"]["participants"][e]["goldEarned"])


for i in range(len(GamesInfo)):
    for e in range(10):
        if GamesInfo[i]["info"]["participants"][e]["role"] == "CARRY":
            CarryExp.append(GamesInfo[i]["info"]["participants"][e]["champExperience"])

In [131]:
somme = sum(CarryExp)
moyenne = somme / len(CarryExp)
print(moyenne)

13145.71336293904
